In [ ]:
import pywt
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.ndimage    
import scipy.signal    
import os



In [ ]:
# read the data: with all subjects: takes a long time (~20min)
if not os.path.exists('../data/training/all_data.csv'):
    strings = ['b','c','d','e','f']
    data = pd.read_csv('../data/training/a_data.csv')
    labels = pd.read_csv('../data/training/a_labels.csv')
    for s in strings:
        print(s)
        temp_data = pd.read_csv('../data/training/'+s+'_data.csv')
        data = pd.concat([data,temp_data])
        temp_labels = pd.read_csv('../data/training/'+s+'_labels.csv')
        labels = pd.concat([labels, temp_labels])
    data.to_csv('../data/training/all_data.csv')
    labels.to_csv('../data/training/all_labels.csv')
else:
    data = pd.read_csv('../data/training/all_data.csv')
    labels = pd.read_csv('../data/training/all_labels.csv')

In [ ]:
# group datapoints into bins, corresponding to a seconnd of recording time
data['TimestampToSec'] = data['Timestamp'].astype(int)
grouped = data.groupby('TimestampToSec')

In [ ]:
def Energy(coeffs, k):
    return np.sqrt(np.sum(np.array(coeffs[-k]) ** 2)) / len(coeffs[-k])

In [ ]:
import pywt
import pywt.data
mode = pywt.Modes.smooth

def signal_decomp(data):
    """Decompose and plot a signal S.
    S = An + Dn + Dn-1 + ... + D1
    """
    w = pywt.Wavelet('db4')
    a = data
    ca = []
    cd = []
    for i in range(5):
        (a, d) = pywt.dwt(a, w, mode)
        ca.append(a)
        cd.append(d)  
    return ca, cd

In [ ]:
import scipy.signal    

# CONSTRUCT FEATURES

# for every label, look up the corresponding data
features = []
for l in range(len(labels)):
    try:
        time = labels['Timestamp'][l]
        slice = grouped.get_group(time)
    except KeyError:
        print(time)
        pass
    # for every channel
    power_all_channels = []
    # 1-7 EEG, 8th channel is ECG data
    for ch in range(8):
        single_sec_ch = slice['Ch{}'.format(ch)]
        
        # median filter the data
        pre_processed = scipy.signal.medfilt(single_sec_ch, kernel_size=3)  
        
        _, cd = signal_decomp(pre_processed)
        # for every decomp. level
        power = []
        for l in range(5):
            power.append(Energy(cd, l))
            
        # collect power for all channels into one vector 
        power_all_channels.append(power) 
    # currently mean power of the frequency bands over all channels are the only features
    power_vec = np.asarray(power_all_channels).flatten()
    features.append(power_vec)
 

In [ ]:
print(np.asarray(features).shape)
target_names = np.unique(labels['Event'])

In [ ]:
import pickle

X_train,y_train,X_val,y_val = train_test_split(
...     features, label['Event'], test_size=0.33, random_state=0)

pickle.dump(X_train,open("../data/training/features_train.p", "wb"))
pickle.dump(y_train,open("../data/training/targets_train.p", "wb"))
pickle.dump(X_val,open("../data/training/features_validation.p", "wb"))
pickle.dump(y_val,open("../data/training/targets_validation.p", "wb"))

In [ ]:
import pickle

X_train =pickle.load(open("../data/training/features_train.p", "rb"))
y_train =pickle.load(open("../data/training/targets_train.p", "rb"))
X_val =pickle.load(open("../data/training/features_validation.p", "rb"))
y_val =pickle.load(open("../data/training/targets_validation.p", "rb"))


In [ ]:
# Utility function to report best scores
def report(results, n_top=n_iter_search):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

In [ ]:
from sklearn import ensemble
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint as sp_randint

# run randomized search
n_iter_search = 20




## Random Forst
clf = ensemble.RandomForestClassifier(n_estimators = 10, criterion='entropy', class_weight='balanced', n_jobs = -1)


# specify parameters and distributions to sample from
param_dist = {"n_estimators":sp_randint(1, 100),
              "max_depth": [3, None],
              "max_features": sp_randint(1, 40),
              "bootstrap": [True, False],
              "criterion": ["gini", "entropy"]}


random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

random_search.fit(X_train, y_train)

report(random_search.cv_results_)
#clf = clf.fit(features, labels['Event'])
#print(clf.feature_importances_)



In [ ]:
#bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'n_estimators': 61

## Random Forst
clf = ensemble.RandomForestClassifier(n_estimators = 73, criterion='entropy', class_weight='balanced', max_features=21, n_jobs = -1)


predicted = cross_val_predict(clf, X_train, y_train, cv=10)

acc = metrics.accuracy_score(y_train, predicted)
print("This is the Score: {}".format(acc))


In [ ]:
pickle.dump(predicted,open("/home/felix/Desktop/forest_predictions.p", "wb"))


In [ ]:
from sklearn.metrics import confusion_matrix
import itertools

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, fontsize=18)
    #plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45, fontsize=18)
    plt.yticks(tick_marks, classes, fontsize=18)

    if normalize:
        float_formatter = lambda x: "%.2f" % x
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, float_formatter(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=18)
        else:
            plt.text(j, i, cm[i, j],
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black", fontsize=18)

    plt.tight_layout()
    plt.ylabel('True label', fontsize=18)
    plt.xlabel('Predicted label', fontsize=18)

In [ ]:
class_names, counts = np.unique(y_train, return_counts=True)
    
# Compute confusion matrix
np.set_printoptions(precision=2)

cnf_matrix = confusion_matrix(y_train, predicted)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

## AdaBoost

In [ ]:
from sklearn import ensemble
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from scipy.stats import randint as sp_randint

# run randomized search
n_iter_search = 20

## AdaBoost
clf = ensemble.AdaBoostClassifier()


# specify parameters and distributions to sample from
param_dist = {"n_estimators":sp_randint(50, 250),
              "algorithm": ["SAMME", "SAMME.R"],
              "base_estimator": [DecisionTreeClassifier(max_depth=1), DecisionTreeClassifier(max_depth=2), DecisionTreeClassifier(max_depth=3)]
             }


random_search = RandomizedSearchCV(clf, param_distributions=param_dist,
                                   n_iter=n_iter_search)

random_search.fit(X_train, y_train)

report(random_search.cv_results_)
#clf = clf.fit(features, labels['Event'])
#print(clf.feature_importances_)



In [ ]:
#bootstrap': True, 'criterion': 'entropy', 'max_depth': None, 'max_features': 10, 'n_estimators': 61

## Random Forst
clf = ensemble.AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=3),n_estimators=188, algorithm ='SAMME.R')


predicted = cross_val_predict(clf, X_train, y_train, cv=10)

acc = metrics.accuracy_score(y_train, predicted)
print("This is the Score: {}".format(acc))

class_names, counts = np.unique(y_train, return_counts=True)
    
# Compute confusion matrix
np.set_printoptions(precision=2)

cnf_matrix = confusion_matrix(y_train, predicted)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

## Validation Set

In [ ]:
forest_clf = ensemble.RandomForestClassifier(n_estimators = 73, criterion='entropy', class_weight='balanced', max_features=21, n_jobs = -1)
forest_clf.fit(X_train,y_train)
forest_pred = forest_clf.predict(X_val)

acc = metrics.accuracy_score(y_val, forest_pred)
print("This is the Score: {}".format(acc))

class_names, counts = np.unique(y_val, return_counts=True)
    
# Compute confusion matrix
np.set_printoptions(precision=2)

cnf_matrix = confusion_matrix(y_val, forest_pred)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix without normalization for validation set')

# Plot normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for validation set')

plt.show()

In [ ]:
ada_clf = ensemble.AdaBoostClassifier(base_estimator= DecisionTreeClassifier(max_depth=3), n_estimators=188, algorithm ='SAMME.R')
ada_clf.fit(X_train,y_train)
ada_pred = ada_clf.predict(X_val)

acc = metrics.accuracy_score(y_val, ada_pred)
print("This is the Score: {}".format(acc))

class_names, counts = np.unique(y_val, return_counts=True)
    
# Compute confusion matrix
np.set_printoptions(precision=2)

cnf_matrix = confusion_matrix(y_val, ada_pred)

# Plot non-normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix without normalization for validation set')

# Plot normalized confusion matrix
plt.figure(figsize=(10,10))
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix for validation set')

plt.show()